# Calculating Activity Onset

## In this notebook, I will:
- import dataset containing multiple spiders
- define a threshold for activity onset for each spider every day
- find the first point that threshold is crossed
- create a dataframe containing all activity onsets

In [2]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

#Read Spider activity CSV file
df = pd.read_csv('Metazygia wittfeldae Monitor 1 Updated_LD.csv',)
df.set_index('Date-Time', inplace = True)

#Turn date-time column into datetime format
df.index = pd.to_datetime(df.index)

df

,Light,Monitor 1 Spider 1,Monitor 1 Spider 2,Monitor 1 Spider 3,Monitor 1 Spider 4,Monitor 1 Spider 5,Monitor 1 Spider 6,Monitor 1 Spider 7,Monitor 1 Spider 8,Monitor 1 Spider 9,...,Monitor 1 Spider 21,Monitor 1 Spider 23,Monitor 1 Spider 25,Monitor 1 Spider 26,Monitor 1 Spider 27,Monitor 1 Spider 28,Monitor 1 Spider 29,Monitor 1 Spider 30,Monitor 1 Spider 31,Monitor 1 Spider 32
Date-Time,,,,,,,,,,,,,,,,,,,,,
2017-04-21 00:00:00,0,0,0,0,0,1,3,0,0,3,...,0,0,0,0,0,2,5,0,0,0
2017-04-21 00:01:00,0,0,0,0,0,0,9,0,0,16,...,0,0,0,0,0,5,10,0,0,0
2017-04-21 00:02:00,0,0,0,0,0,7,2,0,0,10,...,0,0,0,0,4,4,4,0,0,0
2017-04-21 00:03:00,0,0,0,0,0,6,0,0,0,0,...,0,0,0,0,0,5,4,0,0,0
2017-04-21 00:04:00,0,0,0,0,2,2,0,0,0,2,...,0,0,0,0,0,3,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-04-25 23:55:00,0,15,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,2
2017-04-25 23:56:00,0,6,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,10,0
2017-04-25 23:57:00,0,12,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,7,7


### Define necessary functions for calculating activity onset

In [4]:
#This function adds a column to the dataframe with the minute out of the day for that particular row
def add_minutes_column(df):
    df['Total Minutes'] = df.index.minute + (df.index.hour * 60)
    return(df)

#This function is used to create a list of indexes to be used to separate the days
def find_light_switch_index(df):
    transition_time = '07:00:00'
    transition_indexes_df = df.loc[df.index.strftime("%H:%M:%S") == transition_time]
    transition_indexes_list = transition_indexes_df.index.to_list()
    
    return transition_indexes_list

#This function is used in my bigger functions to separate days. Creates a list of dataframes, with each dataframe
#containing a separate day
def create_list_of_df(transition_indexes_list, df):
    list_of_df = []
    for x in range(len(transition_indexes_list)-1):
        list_of_df.append(df[(df.index >= transition_indexes_list[x]) & (df.index < transition_indexes_list[x+1])])
    return(list_of_df)

#This function calculates the rolling average of activity over 30 bins and adds these values as a column of a dataframe
def add_rolling_average_column(df):
    #calculate rolling average of activity
    activity = df.iloc[: , 0]
    rolling = activity.rolling(30).mean().dropna()

    #Add new column to dataframe with the rolling mean
    df['Rolling'] = rolling
    return df